In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder


# Preprocess Data

### Read Images

In [3]:
img = cv2.imread('Ala_Idris\Ala_Idris (1).png')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
data = [img]

label = np.array([0])

for i in range(2,101):
    img = cv2.imread('Ala_Idris\Ala_Idris ({i}).png'.format(i=i))
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [0])

for i in range(1,101):
    img = cv2.imread('Buzgulu\Buzgulu ({i}).png'.format(i=i))
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [1])

for i in range(1,101):
    img = cv2.imread('Dimnit\Dimnit ({i}).png'.format(i=i))
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [2])

for i in range(1,101):
    img = cv2.imread('Nazli/Nazli ({i}).png'.format(i=i))
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [3])

data = data / 255

In [4]:
# label = OneHotEncoder().fit_transform(label.reshape(-1,1)).toarray()

### Split Data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

## Data argumetaions

In [6]:
X_train_arg = X_train.astype(int)
y_train_arg = y_train

#### Horizontal Shift

In [7]:

# datagen = ImageDataGenerator(width_shift_range=[-100,100])
# it = datagen.flow(X_train, batch_size=320,shuffle = False)
# for i in range(1):
# 	batch = it.next()	
# 	# image = batch[1].astype('uint8')
# 	X_train_arg = np.append(X_train_arg, batch.astype(int), axis=0)
# 	y_train_arg = np.append(y_train_arg, y_train, axis=0)




#### Vertical shift

In [8]:
# datagen = ImageDataGenerator(height_shift_range=0.5)
# it = datagen.flow(X_train, batch_size=320,shuffle = False)
# for i in range(1):
# 	batch = it.next()	
# 	# image = batch[1].astype('uint8')
# 	X_train_arg = np.append(X_train_arg, batch.astype(int), axis=0)
# 	y_train_arg = np.append(y_train_arg, y_train, axis=0)



#### 

#### Rotation

In [9]:
# datagen = ImageDataGenerator(rotation_range=90)
# it = datagen.flow(X_train, batch_size=320,shuffle = False)
# for i in range(1):
# 	batch = it.next()	
# 	# image = batch[1].astype('uint8')
# 	X_train_arg = np.append(X_train_arg, batch.astype(int), axis=0)
# 	y_train_arg = np.append(y_train_arg, y_train, axis=0)



## Split Validation

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_arg, y_train_arg, test_size=0.2, random_state=42)

## Auto Encoding

In [12]:
encoding_dim = (224*224)
input_img = keras.Input(shape=(511*511,))

encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
decoded = layers.Dense(783363, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)

encoder = keras.Model(input_img, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# autoencoder.fit(X_train, X_train,
#                 epochs=50,
#                 batch_size=256,
#                 shuffle=False,
#                 validation_data=(X_train, X_valid))

# encoded_imgs = encoder.predict(X_valid)



# Define Network Based on ResNet50

In [13]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [14]:
X_train = preprocess_image_input(X_train)
X_valid = preprocess_image_input(X_valid)


In [15]:
def feature_extractor(inputs):
  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(511, 511,3),
                                              include_top=False,
                                              weights='imagenet')(inputs)
  return feature_extractor

In [16]:
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    # x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(4, activation="softmax", name="classification")(x)
    return x

In [21]:
def final_model(inputs):
    # resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    resize = inputs
    # resnet_feature_extractor = feature_extractor(resize)
    # classification_output = classifier(resnet_feature_extractor)
    classification_output = classifier(resize)
    return classification_output

In [18]:
def compile_model():
    inputs = tf.keras.layers.Input(shape=(511,511,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
    model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
    return model

In [22]:
model = compile_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 511, 511, 3)]     0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 3)                 0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2048      
_________________________________________________________________
classification (Dense)       (None, 4)                 2052      
Total params: 4,100
Trainable params: 4,100
Non-trainable params: 0
_________________________________________________________________


In [24]:
EPOCHS = 4
y_train = y_train.reshape(256,1).astype('uint8')
y_valid = y_valid.reshape(64,1).astype('uint8')
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_data = (X_valid, y_valid), batch_size=8)
# X_train.shape
# y_valid.shape
# print(y_train.shape)
# print(X_valid.shape)
# y_train


Epoch 1/4
32/32 [==============================] - 1s 29ms/step - loss: 1.3859 - accuracy: 0.2617 - val_loss: 1.3886 - val_accuracy: 0.1719
Epoch 2/4
32/32 [==============================] - 1s 19ms/step - loss: 1.3854 - accuracy: 0.2383 - val_loss: 1.3898 - val_accuracy: 0.1719
Epoch 3/4
32/32 [==============================] - 1s 37ms/step - loss: 1.3850 - accuracy: 0.2773 - val_loss: 1.3908 - val_accuracy: 0.1719
Epoch 4/4
32/32 [==============================] - 1s 17ms/step - loss: 1.3846 - accuracy: 0.2773 - val_loss: 1.3918 - val_accuracy: 0.1719
